In [1]:
import pandas as pd
import re

In [2]:
with open("outputs/generated_prompts/20231212-1557_dataset.json", "r") as f:
    data = f.read()

df = pd.read_json(data)
df

,query,description,context,prompt,num_tokens,result,full_answer,is_skipped,is_prompt_too_long
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,"<s>[INST] QUERY=""SELECT ?property ?propertyTyp...",228,"[""Can you retrieve all the numerical propertie...","{'content': ' 1. ""Can you retrieve all the num...",False,False
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,"<s>[INST] QUERY=""SELECT ?id ?idLabel ?idDescri...",261,"[""Can you write a SparQL query to retrieve all...","{'content': ' 1. ""Can you write a SparQL query...",False,False
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,"<s>[INST] QUERY=""SELECT (COUNT(?article) AS ?c...",161,"[""Can you retrieve the number of scientific ar...","{'content': ' 1. ""Can you retrieve the number ...",False,False
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,"<s>[INST] QUERY=""SELECT (COUNT(DISTINCT ?artic...",167,"[""Can you write a SparQL query to count the nu...","{'content': ' 1. ""Can you write a SparQL query...",False,False
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,"<s>[INST] QUERY=""SELECT (COUNT(?item) AS ?coun...",150,"[""Can you write a SparQL query to count the nu...","{'content': ' 1. ""Can you write a SparQL query...",False,False
...,...,...,...,...,...,...,...,...,...
2840,select ?feat ?en (count(*) as ?total)\nwith {\...,Grammatical features used on English nouns and...,,"<s>[INST] QUERY=""select ?feat ?en (count(*) as...",271,"[""Can you write a SparQL query to find the mos...","{'content': ' 1. ""Can you write a SparQL query...",False,False
2841,SELECT ?country ?countryLabel ?lifeExpectancy\...,List of countries sorted by life expectancy,,"<s>[INST] QUERY=""SELECT ?country ?countryLabel...",211,"[""Can you retrieve a list of countries sorted ...","{'content': ' 1. ""Can you retrieve a list of c...",False,False
2842,SELECT ?item ?itemLabel ?coord ?image\nWHERE\n...,Railway bridges in Germany and Austria,,"<s>[INST] QUERY=""SELECT ?item ?itemLabel ?coor...",231,"[""Can you find me all the railway bridges in G...","{'content': ' 1. ""Can you find me all the rail...",False,False
2843,SELECT DISTINCT ?train_wreck ?train_wreckLabel...,Number of deaths in train wrecks per country,,"<s>[INST] QUERY=""SELECT DISTINCT ?train_wreck ...",256,"[""Can you generate a list of train wrecks and ...","{'content': ' 1. ""Can you generate a list of t...",False,False


In [38]:
class MySPARQL():
    def __init__(self, raw_query: str) -> None:
        self.raw_query = raw_query
        self.ent_to_var, self.var_to_ent = self._gen_dict()
        self.template_form = self._templatize()

    def _gen_dict(self):
        matches = re.findall(r"\w+:\w+", self.raw_query)
        
        unique = []
        for x in matches:
            if x not in unique:
                unique.append(x)        
        
        ent_to_var = {match:f"[entity {i}]" for i,match in enumerate(unique)}
        var_to_ent = {value:key for key,value in ent_to_var.items()}
        
        return ent_to_var, var_to_ent
    
    def _templatize(self):
        template_form = self.raw_query
        for key, value in self.ent_to_var.items():
            template_form = template_form.replace(key, value)
        return template_form

In [13]:
query = df['query'][0]
query

'SELECT ?property ?propertyType ?propertyLabel ?propertyDescription WHERE {\n?property wikibase:propertyType ?propertyType .\nSERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }\n} ORDER BY ASC(xsd:integer(STRAFTER(STR(?property), \'P\')))'

In [39]:
new_query = 'SELECT ?property ?propertyType ?propertyLabel ?propertyDescription WHERE {\n?property wikibase:propertyType ?propertyType .\nSERVICE wikibase:label { wikibase:language wikibase:language "[AUTO_LANGUAGE],en". }\n} ORDER BY ASC(xsd:integer(STRAFTER(STR(?property), \'P\')))'
b = MySPARQL(new_query)
print(b.template_form)
print(b.ent_to_var)
print(b.var_to_ent)

SELECT ?property ?propertyType ?propertyLabel ?propertyDescription WHERE {
?property [entity 0] ?propertyType .
SERVICE [entity 1] { [entity 2] [entity 2] "[AUTO_LANGUAGE],en". }
} ORDER BY ASC([entity 3](STRAFTER(STR(?property), 'P')))
{'wikibase:propertyType': '[entity 0]', 'wikibase:label': '[entity 1]', 'wikibase:language': '[entity 2]', 'xsd:integer': '[entity 3]'}
{'[entity 0]': 'wikibase:propertyType', '[entity 1]': 'wikibase:label', '[entity 2]': 'wikibase:language', '[entity 3]': 'xsd:integer'}


In [50]:
def transform(raw_query):
    sparql = MySPARQL(raw_query)
    return sparql.template_form + "\n\n" + "\n".join([f"{key}: {value}" for key, value in sparql.var_to_ent.items()])

df['input'] = df.apply(lambda x: x['result'], axis=1)
df['target'] = df.apply(lambda x: transform(x['query']), axis=1)
df.columns

Index(['query', 'description', 'context', 'prompt', 'num_tokens', 'result',
       'full_answer', 'is_skipped', 'is_prompt_too_long', 'input', 'target'],
      dtype='object')

In [54]:
df[['input', 'target']]

,input,target
0,"[""Can you retrieve all the numerical propertie...",SELECT ?property ?propertyType ?propertyLabel ...
1,"[""Can you write a SparQL query to retrieve all...",SELECT ?id ?idLabel ?idDescription ?new{\n?id ...
2,"[""Can you retrieve the number of scientific ar...",SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...
3,"[""Can you write a SparQL query to count the nu...",SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...
4,"[""Can you write a SparQL query to count the nu...",SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...
...,...,...
2840,"[""Can you write a SparQL query to find the mos...",select ?feat ?en (count(*) as ?total)\nwith {\...
2841,"[""Can you retrieve a list of countries sorted ...",SELECT ?country ?countryLabel ?lifeExpectancy\...
2842,"[""Can you find me all the railway bridges in G...",SELECT ?item ?itemLabel ?coord ?image\nWHERE\n...
2843,"[""Can you generate a list of train wrecks and ...",SELECT DISTINCT ?train_wreck ?train_wreckLabel...


In [52]:
print(df['input'][0])

['"Can you retrieve all the numerical properties on Wikidata, sorted in ascending order?"', '"I need to find all the properties with labels and descriptions on Wikidata, ordered numerically."', '"How can I extract all the properties that have a label and description on Wikidata, and arrange them in numerical order?"']
